In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
tickers = pd.read_csv('./datasets/companylisting.csv', usecols=['ticker']).values

In [3]:
## Get Ratings from Market Watch
ratings = []
for i in tickers:
    r = requests.get('https://www.marketwatch.com/investing/stock/{}/analystestimates'.format(i[0].replace('-','.')))
    soup = bs4.BeautifulSoup(r.text, "html.parser")
    current = soup.findAll('td', {'class': 'current'})
    try:
        ratings.append([i[0], int(current[0].text), int(current[1].text), int(current[2].text), int(current[3].text), int(current[4].text), current[5].text])
    except:
        ratings.append([i[0], np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])

In [14]:
## Import new dataframe into a SQL database
import psycopg2
## Connect to SQL database
conn = psycopg2.connect(database = 'cuzegotk',user = 'cuzegotk',password = 'NekW2BqJ8hW1wO3hCdpuEESPiP-y131V',host = 'raja.db.elephantsql.com')
cur = conn.cursor()
## Transfer CSV to SQL Database
for i in range(0,len(ratings)):
    cur.execute('INSERT INTO analystratings VALUES (%s, %s, %s, %s, %s, %s, %s)', tuple(ratings[i]))
    conn.commit()
cur.close()
conn.close()